In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv', index_col=0)
crypto_df

In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().sum()

In [ ]:
crypto_df = crypto_df[crypto_df['Algorithm'].notnull()]
crypto_df

In [ ]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop('IsTrading', axis=1)
crypto_df

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df

In [ ]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 1]
crypto_df

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = pd.DataFrame(crypto_df['CoinName'])
crypto_names_df

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop('CoinName', axis=1)
crypto_df

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm','ProofType'])
X

In [ ]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
X_scaled

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

In [ ]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data=crypto_pca, columns=['PC 1','PC 2','PC 3'], index=X.index)
pca_df.head(10)

In [ ]:
inertia = []
k = list(range(1,11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

In [ ]:
# Create an elbow curve to find the best value for K.
elbow_data = {'k' : k, 'inertia' : inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x='k', y='inertia', title='Elbow Curve', xticks=k)

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
predictions

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pca_df], axis= 1, join="inner")

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df,crypto_names_df], axis= 1, join="inner")

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color='Class',
    symbol='Class',
    hover_name= 'CoinName',
    hover_data= ['Algorithm'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(
    columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined', 'Class'],
    sortable=True, 
    selectable=True
)

In [ ]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)}  tradable cryptocurrencies.')

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_scaled = MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
clustered_scaled

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data=clustered_scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index.copy()
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']


plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
    by="Class",
    hover_cols="CoinName"
)